In [2]:
import pandas as pd
import numpy as np
from numpy import random

import time

import requests, json
import bs4

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

import re


---
## Fase 1: Inicializar driver y hacer búsqueda
introducir cp, entrar en google y aceptar cookies e ir a gmaps

In [2]:
cp = input('Seleccione un código postal')

Seleccione un código postal28320


In [67]:
def random_sleep():
    time.sleep(random.randint(0,5))

In [ ]:
def drive_start(driver_path):
    service = Service(executable_path=driver_path)
    driver = webdriver.Firefox(service=service)
    return driver

In [69]:
random_sleep()

In [ ]:
def gmaps_search(site, cp):
    return f'https://www.google.com/search?q={site}+en+{cp}&rlz=1C5CHFA_enES1019ES1019&oq={site}+en+{cp}&aqs=chrome..69i57j69i60l3.4633j0j15&sourceid=chrome&ie=UTF-8'

In [ ]:
def go_to_maps(url, driver):
    try: 
        driver.get(url)
        driver.maximize_window()
        cookie_button = driver.find_element(by=By.ID, value='L2AGLb')
        cookie_button.click()
        random_sleep()
        mapslink = driver.find_element(By.XPATH, '/html/body/div[8]/div/div[4]/div/div[1]/div/div[1]/div/div[2]/a')
        mapslink.click()
        return True
    except:
        return False
    

In [3]:
path = '../resources/firefox_driver/geckodriver'
service = Service(executable_path=path)
driver = webdriver.Firefox(service=service)

In [4]:
url = f'https://www.google.com/search?q=restaurantes+en+{cp}&rlz=1C5CHFA_enES1019ES1019&oq=restaurantes+en+28031&aqs=chrome..69i57j69i60l3.4633j0j15&sourceid=chrome&ie=UTF-8'

In [5]:
driver.get(url)
driver.maximize_window()

In [6]:
cookie_button = driver.find_element(by=By.ID, value='L2AGLb')
cookie_button

<selenium.webdriver.remote.webelement.WebElement (session="df6d58fc-0170-4b6f-a06e-2a0688fbe6c7", element="f0bb0c6f-4332-411e-87f7-3d17249931c6")>

In [7]:
cookie_button.click()

In [8]:
time.sleep(3)

In [9]:
mapslink = driver.find_element(By.XPATH, '/html/body/div[8]/div/div[4]/div/div[1]/div/div[1]/div/div[2]/a')
mapslink.click()

---
## Fase 2: hacer scroll para cargar todos los lugares y obtenerlos

In [10]:
def full_scroll(driver):
    
    
    altura = driver.execute_script('return document.querySelector(".ecceSd").firstChild.scrollHeight')

    nueva_altura = 0
    while True:   

        driver.execute_script(f'document.querySelector(".ecceSd ").firstChild.scrollTo(0, {altura});')

        random_sleep()
        
        nueva_altura = driver.execute_script('return document.querySelector(".ecceSd").firstChild.firstChild.scrollHeight')
    
        altura = nueva_altura
        if altura == nueva_altura:
            break


In [ ]:
def scroll_places(driver):
    while True:
        try:
            driver.find_element(By.CLASS_NAME, 'HlvSq')
            print('final')
            break
        except:
            full_scroll(driver) 
    

In [ ]:
def get_all_cards(driver):
    try:   
        cards = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
        random_sleep()
        return cards
    except:
        return False

In [11]:
while True:
    try:
        driver.find_element(By.CLASS_NAME, 'HlvSq')
        print('final')
        break
    except:
        full_scroll(driver) 

final


In [12]:
cards = driver.find_elements(By.CLASS_NAME, 'hfpxzc')

In [13]:
time.sleep(3)

---
## Fase 3: Recorrer lugar por lugar y scrapear datos

In [ ]:
def cards_travel(driver, cards):
    nombres = []
    rates = []
    nums_resenas = []
    pricings = []
    detalles = []
    labels = []
    descripciones = []
    urls = [i.get_attribute('href') for i in cards]

    for i in range(len(cards)):
        cards[i].click()

        random_sleep()

        try:
            nombre = driver.find_element(By.CLASS_NAME, 'DUwDvf').text
            nombres.append(nombre)
        except:
            nombres.append('')

        try:
            rate = driver.find_element(By.CLASS_NAME, 'fontDisplayLarge').text
            rates.append(rate)
        except:
            rates.append('')

        try:
            num_resenas = driver.find_element(By.CLASS_NAME, 'DkEaL').text
            nums_resenas.append(num_resenas)

        except:
            num_resenas.append('')

        try:
            pricing = driver.find_element(By.CLASS_NAME, 'mgr77e').text
            pricings.append(pricing)
        except:
            pricings.append('')

        try:
            detalle = driver.find_elements(By.CLASS_NAME, 'rogA2c')
            detalles.append([i.text for i in detalle])
        except:
            detalles.append('')

        try:
            label = driver.find_element(By.CLASS_NAME,'u6ijk').text
            labels.append(label)
        except:
            labels.append('')
        try:
            descripcion = driver.find_element(By.CLASS_NAME,'PYvSYb').text
            descripciones.append(descripcion)
        except:
            descripciones.append('')
    
    return nombres, rates, nums_resenas, detalles, pricings, labels, descripciones, urls     

In [14]:
nombres = []
rates = []
nums_resenas = []
pricings = []
detalles = []
labels = []
descripciones = []


for i in range(len(cards)):
    cards[i].click()
    
    time.sleep(1)
    
    
    try:
        nombre = driver.find_element(By.CLASS_NAME, 'DUwDvf').text
        nombres.append(nombre)
    except:
        nombres.append('')
        
    try:
        rate = driver.find_element(By.CLASS_NAME, 'fontDisplayLarge').text
        rates.append(rate)
    except:
        rates.append('')

    try:
        num_resenas = driver.find_element(By.CLASS_NAME, 'DkEaL').text
        nums_resenas.append(num_resenas)
        
    except:
        num_resenas.append('')
    
    try:
        pricing = driver.find_element(By.CLASS_NAME, 'mgr77e').text
        pricings.append(pricing)
    except:
        pricings.append('')
        
    try:
        detalle = driver.find_elements(By.CLASS_NAME, 'rogA2c')
        detalles.append([i.text for i in detalle])
    except:
        detalles.append('')
        
    try:
        label = driver.find_element(By.CLASS_NAME,'u6ijk').text
        labels.append(label)
    except:
        labels.append('')
    try:
        descripcion = driver.find_element(By.CLASS_NAME,'PYvSYb').text
        descripciones.append(descripcion)
    except:
        descripciones.append('')
        
    
    
        

In [15]:
urls = [i.get_attribute('href') for i in cards]

In [16]:
urls

['https://www.google.com/maps/place/Restaurante+La+Suite+del+Lago/data=!4m7!3m6!1s0xd421f71ed32299b:0xdf19860a461ac8f8!8m2!3d40.2357902!4d-3.6940003!16s%2Fg%2F12cp1dd92!19sChIJmyky7XEfQg0R-MgaRgqGGd8?authuser=0&hl=es&rclk=1',
 'https://www.google.com/maps/place/Mes%C3%B3n+Toledano/data=!4m7!3m6!1s0xd421f7dd15dd4ff:0xf724151355ba83d8!8m2!3d40.2468351!4d-3.6974291!16s%2Fg%2F1tf6ynhq!19sChIJ_9Rd0X0fQg0R2IO6VRMVJPc?authuser=0&hl=es&rclk=1',
 'https://www.google.com/maps/place/La+Terracita/data=!4m7!3m6!1s0xd421f825ea6cc07:0xcdf76aae2e99ad23!8m2!3d40.2435188!4d-3.7044401!16s%2Fg%2F11b6yjmr0k!19sChIJB8ymXoIfQg0RI62ZLq5q980?authuser=0&hl=es&rclk=1',
 'https://www.google.com/maps/place/El+Barril/data=!4m7!3m6!1s0xd421f798fa35b1f:0xe2d3a2eca0d43048!8m2!3d40.2416096!4d-3.6981716!16s%2Fg%2F11bx5q0fd7!19sChIJH1ujj3kfQg0RSDDUoOyi0-I?authuser=0&hl=es&rclk=1',
 'https://www.google.com/maps/place/Puerto+Pirata/data=!4m7!3m6!1s0xd421f7b9240feb5:0xf426d2d6b5135368!8m2!3d40.2441816!4d-3.6963289!16s%2Fg%2

---
## Fase 4: Scraping terminado. Cerramos driver

In [ ]:
def finish_driver(driver):
    driver.quit()
    return True

In [17]:
driver.quit()

# Hasta aquí Selenium :D
---

## Ahora hay que extraer cada dato de la lista
---
1. Direccion siempre es la posición 0 de la lista
2. Teléfono necesita REGEX siempre tiene formato 3 2 2 2
3. Web necesita regex con .com o .es Hay que filtrar los menus y los hacer un pedido en just eat


In [18]:
rates

['4,3',
 '4,4',
 '4,4',
 '4,1',
 '4,2',
 '4,7',
 '4,0',
 '3,8',
 '4,2',
 '4,2',
 '3,9',
 '4,6',
 '3,9',
 '2,5',
 '4,6',
 '4,3',
 '4,3',
 '4,0',
 '4,3',
 '4,3',
 '4,8',
 '4,8',
 '4,5',
 '4,5',
 '4,9',
 '4,1',
 '4,5',
 '4,1',
 '3,6',
 '',
 '4,4',
 '4,2',
 '4,2',
 '4,2',
 '3,5',
 '3,9',
 '4,3',
 '4,4',
 '4,3',
 '4,6',
 '5,0',
 '4,3',
 '4,5',
 '',
 '4,4',
 '4,5',
 '4,3',
 '4,1',
 '4,0',
 '4,0',
 '4,2',
 '4,1',
 '4,3',
 '4,5',
 '4,2',
 '3,6',
 '3,6',
 '3,8',
 '3,4',
 '4,0',
 '4,3',
 '4,8',
 '4,3',
 '4,5',
 '4,4',
 '4,0',
 '4,6',
 '4,5',
 '4,4',
 '3,9',
 '3,9',
 '3,3',
 '3,8',
 '3,0',
 '4,0',
 '4,0',
 '4,7',
 '3,9',
 '4,1',
 '4,2']

In [19]:
ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79]

In [20]:
nombres

['Restaurante La Suite del Lago',
 'Mesón Toledano',
 'La Terracita',
 'El Barril',
 'Puerto Pirata',
 "Garri's",
 'Restaurante el Almendro Pinto',
 'La Posada',
 'El Buen Yantar de Azucena',
 'Mesón Salazar',
 "My'Secret",
 'La Cúspide Pinto - Restaurante Peruano',
 'Mag Café',
 'Restaurante El Buen Comer',
 'RESTAURANTE EVELIN',
 'Las Patatas de Javi',
 'Il Vit Restaurante Cafetería',
 'Gela',
 'Crokante',
 'Eneria',
 'Reverso Restaurante',
 'Asador de Pollos El Buen Gusto',
 'Malauva',
 'A Fuego Lento',
 'Restaurante Nadu',
 'El Asador De Pinto',
 'El Teatro de Baco',
 'La Barbería',
 'La Capilla Del Parque',
 'Restaurante Maica',
 'Asador ¡Cómo como!',
 'El Apretón II pinto',
 'Cafetería Romayka de David Aguilar',
 'La Esperanza',
 'Restaurante LIPARI.',
 'Jamonivino Restaurante Cerveceria',
 'La Tav’herna',
 'Los Ratitos de Marotti',
 'OnSushi',
 'Ponzano',
 'Como la trucha al trucho',
 'Cervecería Atila',
 'Restaurante Kuala',
 'Al Barrio',
 'La Ostra',
 'Anteux Burger',
 'Cafete

In [21]:
nums_resenas

['2.723 reseñas',
 '804 reseñas',
 '220 reseñas',
 '359 reseñas',
 '1.060 reseñas',
 '165 reseñas',
 '147 reseñas',
 '313 reseñas',
 '92 reseñas',
 '101 reseñas',
 '650 reseñas',
 '339 reseñas',
 '519 reseñas',
 '8 reseñas',
 '14 reseñas',
 '496 reseñas',
 '108 reseñas',
 '490 reseñas',
 '311 reseñas',
 '83 reseñas',
 '292 reseñas',
 '427 reseñas',
 '223 reseñas',
 '167 reseñas',
 '19 reseñas',
 '202 reseñas',
 '142 reseñas',
 '886 reseñas',
 '716 reseñas',
 'Restaurante',
 '112 reseñas',
 '704 reseñas',
 '410 reseñas',
 '154 reseñas',
 '343 reseñas',
 '89 reseñas',
 '74 reseñas',
 '210 reseñas',
 '280 reseñas',
 '107 reseñas',
 '27 reseñas',
 '2.183 reseñas',
 '30 reseñas',
 'Bar restaurante',
 '321 reseñas',
 '534 reseñas',
 '491 reseñas',
 '120 reseñas',
 '567 reseñas',
 '201 reseñas',
 '355 reseñas',
 '450 reseñas',
 '587 reseñas',
 '83 reseñas',
 '447 reseñas',
 '1.281 reseñas',
 '376 reseñas',
 '239 reseñas',
 '3.118 reseñas',
 '413 reseñas',
 '123 reseñas',
 '65 reseñas',
 '114 

### Cleaning resenas

In [22]:
def resenas_cleaning(resenas):
    cleaned_list = []
    for i in resenas:
        i = i.replace('.', '')

        match = re.search('\d*', i)
        cleaned_list.append(match.group(0))

    return cleaned_list

In [26]:

def pricing_cleaning(pricings):
    cleaned_list = []
    for i in pricings:
        
        match = re.search('€*$', i)
        cleaned_list.append(match.group(0))
        
    return cleaned_list

In [23]:
nums_resenas = resenas_cleaning(nums_resenas)

In [24]:
nums_resenas

['2723',
 '804',
 '220',
 '359',
 '1060',
 '165',
 '147',
 '313',
 '92',
 '101',
 '650',
 '339',
 '519',
 '8',
 '14',
 '496',
 '108',
 '490',
 '311',
 '83',
 '292',
 '427',
 '223',
 '167',
 '19',
 '202',
 '142',
 '886',
 '716',
 '',
 '112',
 '704',
 '410',
 '154',
 '343',
 '89',
 '74',
 '210',
 '280',
 '107',
 '27',
 '2183',
 '30',
 '',
 '321',
 '534',
 '491',
 '120',
 '567',
 '201',
 '355',
 '450',
 '587',
 '83',
 '447',
 '1281',
 '376',
 '239',
 '3118',
 '413',
 '123',
 '65',
 '114',
 '399',
 '261',
 '155',
 '653',
 '105',
 '148',
 '174',
 '8',
 '137',
 '111',
 '3',
 '1377',
 '205',
 '273',
 '204',
 '202',
 '413']

In [25]:
pricings
# NECESITA LIMPIEZA

['·€€',
 '·€€',
 '·€',
 '',
 '·€',
 '',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '·€',
 '',
 '',
 '',
 '·€',
 '',
 '·€',
 '',
 '·€€',
 '',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '',
 '·€',
 '·€€',
 '',
 '',
 '·€',
 '',
 '',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '·€',
 '',
 '·€€',
 '·€',
 '·€',
 '·€',
 '·€',
 '',
 '·€',
 '',
 '·€',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '·€',
 '·€',
 '',
 '·€',
 '·€',
 '',
 '·€',
 '',
 '',
 '·€',
 '',
 '·€']

In [27]:
pricings = pricing_cleaning(pricings)


In [28]:
pricings

['€€',
 '€€',
 '€',
 '',
 '€',
 '',
 '€',
 '€',
 '€',
 '€',
 '€',
 '',
 '€',
 '€',
 '',
 '€',
 '€',
 '€',
 '',
 '',
 '',
 '€',
 '',
 '€',
 '',
 '€€',
 '',
 '€',
 '€',
 '',
 '€',
 '€',
 '€',
 '',
 '€',
 '€',
 '',
 '€',
 '€€',
 '',
 '',
 '€',
 '',
 '',
 '€',
 '€',
 '€',
 '€',
 '€',
 '€',
 '€',
 '€',
 '€',
 '',
 '€€',
 '€',
 '€',
 '€',
 '€',
 '',
 '€',
 '',
 '€',
 '€',
 '€',
 '',
 '€',
 '€',
 '€',
 '€',
 '',
 '€',
 '€',
 '',
 '€',
 '',
 '',
 '€',
 '',
 '€']

In [29]:
labels

['Restaurante',
 'Restaurante mediterráneo',
 'Bar restaurante',
 'Restaurante',
 'Restaurante',
 'Hamburguesería',
 'Bar restaurante',
 'Bar restaurante',
 'Restaurante especializado en pollo',
 'Restaurante',
 'Restaurante',
 'Restaurante peruano',
 'Restaurante',
 'Restaurante',
 'Bar restaurante',
 'Restaurante',
 'Restaurante',
 'Cervecería',
 'Restaurante',
 'Restaurante',
 'Restaurante',
 'Restaurante especializado en pollo',
 'Restaurante',
 'Restaurante de comida para llevar',
 'Restaurante',
 'Bar restaurante',
 'Restaurante',
 'Bar',
 'Restaurante',
 '',
 'Restaurante especializado en pollo',
 'Bar',
 'Cafetería',
 'Bar restaurante',
 'Hamburguesería',
 'Restaurante',
 'Restaurante',
 'Cafetería',
 'Restaurante de sushi',
 'Bar restaurante',
 'Restaurante',
 'Bar de tapas',
 'Restaurante',
 '',
 'Pub',
 'Hamburguesería',
 'Bar',
 'Restaurante',
 'Bar restaurante',
 'Bar restaurante',
 'Bar',
 'Restaurante asiático',
 'Pizzería',
 'Restaurante',
 'Marisquería',
 'Bar de tapas

In [30]:
descripciones

['Platos creativos en un luminoso restaurante con vistas al agua y en terraza con césped artificial y sofás.',
 'Bodega a la vista y pared de ladrillo en restaurante especializado en carnes a la brasa con guisos de mercado.',
 'Tapas y platos típicos componen el menú de este restaurante sencillo de estilo clásico.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Compacto salón formal y acogedor de mantel blanco que sirve platos de parrilla, asados y pulpo a la gallega.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Veterano y compacto local de comida casera para llevar especializado en el asado de pollos y croquetas.',
 '',
 '',
 '',
 'Cocina tradicional, sidra y pinchos vascos en un asador de amplios salones clásicos para celebraciones.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Platos típicos orientales y menú degustación en un curioso salón con ventanales, parque infantil y terraza.',
 '',
 '',
 '',
 

In [ ]:
def get_direcciones(detalles):
    direcciones = [i[0] for i in detalles]
    return direcciones

In [32]:
def phone_numbers(detalles):

    final_list = []
    for i in detalles:
        for j in i:
            match = re.search('(\d*) (\d*) (\d*) (\d*)', j)
            try:
                final_list.append([i[0], match.group(0)])
            except:
                final_list.append([i[0], None])
    phones_df = pd.DataFrame(final_list)
    phones_df.dropna(inplace=True)
    phones_df = phones_df.rename(columns={0:'direccion', 1:'numero'})
    print(phones_df)
    return phones_df
            

In [52]:
def create_restaurantes_df(nombres, rates, nums_resenas, pricings, labels, descripciones, direcciones, urls, phone_numbers_df):
    data = {
        'nombre': nombres,
        'rate': rates,
        'resenas': nums_resenas,
        'precio': pricings,
        'labels': labels,
        'descripcion': descripciones,
        'direccion':direcciones,
        'url': urls
    }
    restaurantes_df = pd.DataFrame(data)
    restaurantes_df = pd.merge(restaurantes_df,phone_numbers_df,on='direccion', how='left')
    restaurantes_df = restaurantes_df.fillna('-')

    return restaurantes_df

In [31]:
direcciones = [i[0] for i in detalles]
direcciones

['Parque Juan Carlos I, C. Pablo Picasso, 2, 28320 Pinto, Madrid',
 'C. de Castilla, 13, 28320 Pinto, Madrid',
 'C. de Fuentevieja, 2, 28320 Pinto, Madrid',
 'C. de la Nación Dominicana, 22, 28320 Pinto, Madrid',
 'C. de Pedro Rubin de Celis, 19, 28320 Pinto, Madrid',
 'Av. Juan Pablo II, 18, local 5, 28320 Pinto, Madrid',
 'C. Milanos, 10, 28320 Pinto, Madrid',
 'P.º de Dolores Soria, 2, 28320 Pinto, Madrid',
 'C. Manuel de Falla, local 4, 28320 Pinto, Madrid',
 'C. Emilio Zubiría, 24, 28320 Pinto, Madrid',
 'Av. Juan Pablo II, 22, 28320 Pinto, Madrid',
 'C. Urogallos, 16, Nave 7, 28320 Pinto, Madrid',
 'C. del Pajar, 9, 28320 Pinto, Madrid',
 'C. Milanos, 6, 28320 Pinto, Madrid',
 'C. de la Empedrada, 7, 28320 Pinto, Madrid',
 'C. Alfaro, 9, 28320 Pinto, Madrid',
 'C. Buenavista, 5, 28320 Pinto, Madrid',
 'C. Joan Miró, 14, 28320 Pinto, Madrid',
 'C. de San Juan, 18, 28320 Pinto, Madrid',
 'C. Manuel de Falla, 11, 28320 Pinto, Madrid',
 'C. de Ana de Austria, 1, Local 11, 28320 Pinto

In [33]:
len(phone_numbers(detalles))

                                             direccion        numero
2    Parque Juan Carlos I, C. Pablo Picasso, 2, 283...  910 00 25 82
7              C. de Castilla, 13, 28320 Pinto, Madrid  916 91 45 37
12           C. de Fuentevieja, 2, 28320 Pinto, Madrid  916 92 62 02
16   C. de la Nación Dominicana, 22, 28320 Pinto, M...  916 91 55 92
23   C. de Pedro Rubin de Celis, 19, 28320 Pinto, M...  912 27 43 51
..                                                 ...           ...
404                 C. Alfaro, 16, 28320 Pinto, Madrid  686 75 61 47
410            Pl. de Santiago, 1, 28320 Pinto, Madrid  910 00 56 88
414               C. Valdemoro, 3, 28320 Pinto, Madrid  916 92 26 05
421       Av. de Antonio López, 5, 28320 Pinto, Madrid  625 56 71 35
427         Av. Juan Pablo II, 18, 28320 Pinto, Madrid  915 45 60 31

[69 rows x 2 columns]


69

In [34]:
phone_numbers_df = phone_numbers(detalles)
phone_numbers_df

                                             direccion        numero
2    Parque Juan Carlos I, C. Pablo Picasso, 2, 283...  910 00 25 82
7              C. de Castilla, 13, 28320 Pinto, Madrid  916 91 45 37
12           C. de Fuentevieja, 2, 28320 Pinto, Madrid  916 92 62 02
16   C. de la Nación Dominicana, 22, 28320 Pinto, M...  916 91 55 92
23   C. de Pedro Rubin de Celis, 19, 28320 Pinto, M...  912 27 43 51
..                                                 ...           ...
404                 C. Alfaro, 16, 28320 Pinto, Madrid  686 75 61 47
410            Pl. de Santiago, 1, 28320 Pinto, Madrid  910 00 56 88
414               C. Valdemoro, 3, 28320 Pinto, Madrid  916 92 26 05
421       Av. de Antonio López, 5, 28320 Pinto, Madrid  625 56 71 35
427         Av. Juan Pablo II, 18, 28320 Pinto, Madrid  915 45 60 31

[69 rows x 2 columns]


,direccion,numero
2,"Parque Juan Carlos I, C. Pablo Picasso, 2, 283...",910 00 25 82
7,"C. de Castilla, 13, 28320 Pinto, Madrid",916 91 45 37
12,"C. de Fuentevieja, 2, 28320 Pinto, Madrid",916 92 62 02
16,"C. de la Nación Dominicana, 22, 28320 Pinto, M...",916 91 55 92
23,"C. de Pedro Rubin de Celis, 19, 28320 Pinto, M...",912 27 43 51
...,...,...
404,"C. Alfaro, 16, 28320 Pinto, Madrid",686 75 61 47
410,"Pl. de Santiago, 1, 28320 Pinto, Madrid",910 00 56 88
414,"C. Valdemoro, 3, 28320 Pinto, Madrid",916 92 26 05
421,"Av. de Antonio López, 5, 28320 Pinto, Madrid",625 56 71 35


In [35]:
direcciones

['Parque Juan Carlos I, C. Pablo Picasso, 2, 28320 Pinto, Madrid',
 'C. de Castilla, 13, 28320 Pinto, Madrid',
 'C. de Fuentevieja, 2, 28320 Pinto, Madrid',
 'C. de la Nación Dominicana, 22, 28320 Pinto, Madrid',
 'C. de Pedro Rubin de Celis, 19, 28320 Pinto, Madrid',
 'Av. Juan Pablo II, 18, local 5, 28320 Pinto, Madrid',
 'C. Milanos, 10, 28320 Pinto, Madrid',
 'P.º de Dolores Soria, 2, 28320 Pinto, Madrid',
 'C. Manuel de Falla, local 4, 28320 Pinto, Madrid',
 'C. Emilio Zubiría, 24, 28320 Pinto, Madrid',
 'Av. Juan Pablo II, 22, 28320 Pinto, Madrid',
 'C. Urogallos, 16, Nave 7, 28320 Pinto, Madrid',
 'C. del Pajar, 9, 28320 Pinto, Madrid',
 'C. Milanos, 6, 28320 Pinto, Madrid',
 'C. de la Empedrada, 7, 28320 Pinto, Madrid',
 'C. Alfaro, 9, 28320 Pinto, Madrid',
 'C. Buenavista, 5, 28320 Pinto, Madrid',
 'C. Joan Miró, 14, 28320 Pinto, Madrid',
 'C. de San Juan, 18, 28320 Pinto, Madrid',
 'C. Manuel de Falla, 11, 28320 Pinto, Madrid',
 'C. de Ana de Austria, 1, Local 11, 28320 Pinto

In [53]:
restaurantes_df = create_restaurantes_df(ids, nombres, rates, nums_resenas, pricings, labels, descripciones, direcciones,urls, phone_numbers_df)
restaurantes_df

,nombre,rate,resenas,precio,labels,descripcion,direccion,url,numero
0,Restaurante La Suite del Lago,"4,3",2723,€€,Restaurante,Platos creativos en un luminoso restaurante co...,"Parque Juan Carlos I, C. Pablo Picasso, 2, 283...",https://www.google.com/maps/place/Restaurante+...,910 00 25 82
1,Mesón Toledano,"4,4",804,€€,Restaurante mediterráneo,Bodega a la vista y pared de ladrillo en resta...,"C. de Castilla, 13, 28320 Pinto, Madrid",https://www.google.com/maps/place/Mes%C3%B3n+T...,916 91 45 37
2,La Terracita,"4,4",220,€,Bar restaurante,Tapas y platos típicos componen el menú de est...,"C. de Fuentevieja, 2, 28320 Pinto, Madrid",https://www.google.com/maps/place/La+Terracita...,916 92 62 02
3,El Barril,"4,1",359,,Restaurante,,"C. de la Nación Dominicana, 22, 28320 Pinto, M...",https://www.google.com/maps/place/El+Barril/da...,916 91 55 92
4,Puerto Pirata,"4,2",1060,€,Restaurante,,"C. de Pedro Rubin de Celis, 19, 28320 Pinto, M...",https://www.google.com/maps/place/Puerto+Pirat...,912 27 43 51
...,...,...,...,...,...,...,...,...,...
79,La cantina de Frida,"4,0",205,,Restaurante mexicano,,"C. Alfaro, 16, 28320 Pinto, Madrid",https://www.google.com/maps/place/La+cantina+d...,686 75 61 47
80,Restaurante Panpintao,"4,7",273,,Restaurante,,"Pl. de Santiago, 1, 28320 Pinto, Madrid",https://www.google.com/maps/place/Restaurante+...,910 00 56 88
81,Jamon Y Vino,"3,9",204,€,Restaurante,,"C. Valdemoro, 3, 28320 Pinto, Madrid",https://www.google.com/maps/place/Jamon+Y+Vino...,916 92 26 05
82,Bar Central Tenería,"4,1",202,,Pub,,"Av. de Antonio López, 5, 28320 Pinto, Madrid",https://www.google.com/maps/place/Bar+Central+...,625 56 71 35


---
## Grabar datos en bd

In [3]:
from sqlalchemy import create_engine

In [ ]:
def db_connection(path):
    engine = create_engine(path)
    connection = engine.connect()
    return connection

In [ ]:
def append_to_sql(table_name, connection):
    try:
        restaurantes_df.to_sql(table_name, connection, if_exists='append', index=False)
        return 'Data saved succesfully.'
    except:
        return 'There was a problem saving data.'

In [4]:
engine = create_engine(f'sqlite:///../../../ih_final_project_DB/ih_final_project') 
con = engine.connect()

nombre_tabla = engine.table_names()

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_25741/355435169.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  nombre_tabla = engine.table_names()


In [5]:
print(nombre_tabla)

['activity_tracking', 'credenciales_usuarios', 'eventos', 'preferencias_generales', 'preferencias_restaurantes', 'restaurantes', 'sqlite_sequence', 'usuarios']


In [21]:
restaurantes_df.to_sql('restaurantes', con, if_exists='append', index=False)

NameError: name 'restaurantes_df' is not defined

In [28]:
rest = pd.read_sql('SELECT * FROM restaurantes', con)

In [29]:
rest.tail(50)

,id,nombre,rate,resenas,precio,labels,descripcion,direccion,url,numero
70,373,Asador ¡Cómo como!,"4,4",112,€,Restaurante especializado en pollo,,"C. San Emilio, 2, 28320 Pinto, Madrid",https://www.google.com/maps/place/Asador+%C2%A...,911 48 64 21
71,374,Asador de Pollos El Buen Gusto,"4,8",427,€,Restaurante especializado en pollo,Veterano y compacto local de comida casera par...,"C/ San Vicente Martir 1, 28320 Pinto, Madrid",https://www.google.com/maps/place/Asador+de+Po...,916 91 67 04
72,375,kebab express & fried chicken,"3,3",43,,Restaurante de comida rápida,,"para domicilio 910816916 O 632776951, Av. de I...",https://www.google.com/maps/place/kebab+expres...,632 77 69 51
73,376,El Apeadero de Pinto,"4,3",167,,Restaurante,,"C. de Buenos Aires, 2, Local 10 planta 1, 2832...",https://www.google.com/maps/place/El+Apeadero+...,650 32 91 72
74,377,Cafetería Romayka de David Aguilar,"4,2",412,€,Cafetería,,"Av. de Antonio López, 19, 28320 Pinto, Madrid",https://www.google.com/maps/place/Cafeter%C3%A...,669 67 80 18
75,378,Manhattan Lounge,"4,5",98,,Pub,,"Av. de Antonio López, 13, 28320 Pinto, Madrid",https://www.google.com/maps/place/Manhattan+Lo...,911 56 67 66
76,379,Foster's Hollywood,"3,3",854,€€,Hamburguesería,Hamburguesas y comida típica americana y tex-m...,"C.C. Plaza Eboli, C. Pablo Picasso, 10, 28320 ...",https://www.google.com/maps/place/Foster's+Hol...,916 92 79 61
77,380,Restaurante LIPARI.,"3,5",343,€,Hamburguesería,,"C. Sur, 10, 28320 Pinto, Madrid",https://www.google.com/maps/place/Restaurante+...,636 27 43 21
78,381,Restaurante el Cisne,"2,3",3,,Restaurante,,"C. de Ana de Austria, 11, 28320 Pinto, Madrid",https://www.google.com/maps/place/Restaurante+...,916 92 18 31
79,382,La Tav’herna,"4,3",74,,Restaurante,,"Av. de Isabel la Católica, 25, 28320 Pinto, Ma...",https://www.google.com/maps/place/La+Tav%E2%80...,-


In [30]:
rest['nombre'].value_counts()

Bar Silva                          2
Taqueria Echale Compadre           2
Malauva                            2
Eneria                             2
Donde ling asian cuisine           2
                                  ..
Oasis Cafetería                    1
La Regina                          1
McDonald's                         1
La Bahia de Cadiz                  1
Restaurante Chino Nueva Muralla    1
Name: nombre, Length: 112, dtype: int64

In [19]:
rest = rest.drop_duplicates(subset=['nombre'])

In [20]:
rest

,id,nombre,rate,resenas,precio,labels,descripcion,direccion,url,numero
0,159,La Terracita,"4,4",220,€,Bar restaurante,Tapas y platos típicos componen el menú de est...,"C. de Fuentevieja, 2, 28320 Pinto, Madrid",https://www.google.com/maps/place/La+Terracita...,916 92 62 02
1,160,Puerto Pirata,"4,2",1062,€,Restaurante,,"C. de Pedro Rubin de Celis, 19, 28320 Pinto, M...",https://www.google.com/maps/place/Puerto+Pirat...,912 27 43 51
2,161,El Barril,"4,1",359,,Restaurante,,"C. de la Nación Dominicana, 22, 28320 Pinto, M...",https://www.google.com/maps/place/El+Barril/da...,916 91 55 92
3,162,Garri's,"4,7",168,,Hamburguesería,,"Av. Juan Pablo II, 18, local 5, 28320 Pinto, M...",https://www.google.com/maps/place/Garri's/data...,914 32 21 36
4,163,La Posada,"3,8",314,€,Bar restaurante,,"P.º de Dolores Soria, 2, 28320 Pinto, Madrid",https://www.google.com/maps/place/La+Posada/da...,-
5,164,My'Secret,"3,9",653,€,Restaurante,,"Av. Juan Pablo II, 22, 28320 Pinto, Madrid",https://www.google.com/maps/place/My'Secret/da...,911 37 83 59
6,165,Mag Café,"3,9",521,€,Restaurante,Compacto salón formal y acogedor de mantel bla...,"C. del Pajar, 9, 28320 Pinto, Madrid",https://www.google.com/maps/place/Mag+Caf%C3%A...,916 92 41 38
7,166,Crokante,"4,3",312,,Restaurante,,"C. de San Juan, 18, 28320 Pinto, Madrid",https://www.google.com/maps/place/Crokante/dat...,917 82 29 36
8,167,Restaurante Nadu,"4,9",19,,Restaurante,,"C. de San José, 3, 28320 Pinto, Madrid",https://www.google.com/maps/place/Restaurante+...,618 27 96 66
9,168,La Barbería,"4,1",886,€,Bar,,"Av. Juan Pablo II, 74, 28320 Pinto, Madrid",https://www.google.com/maps/place/La+Barber%C3...,691 77 72 13
